In [1]:
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [8]:
a = unpickle('CIFAR_Data/cifar-10-batches-py/data_batch_1')
print(a.keys()) # we only need labels and data

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [11]:
print(np.array(a[b'labels']).shape)
print(a[b'data'].shape) # every photo's shape is [32,32,3]

(10000,)
(10000, 3072)


In [17]:
# data preprocessing
file_name = 'CIFAR_Data/cifar-10-batches-py/'
x_train = np.zeros((50000,32,32,3))
y_train = np.zeros(50000)
x_test = np.zeros((10000,32,32,3))
y_test = np.zeros(10000)
for i in range(5):
    batch_dict = unpickle(file_name+'data_batch_'+str(i+1))
    batch_data = np.array(batch_dict[b'data'],dtype = np.float32).reshape(10000,32,32,3)
    batch_label = np.array(batch_dict[b'labels'])
    
    x_train[i*10000:(i+1)*10000] = batch_data
    y_train[i*10000:(i+1)*10000] = batch_label

test_dict = unpickle(file_name+'test_batch')
x_test = np.array(test_dict[b'data'],dtype = np.float32).reshape(10000,32,32,3)
y_test = np.array(test_dict[b'labels'])

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [18]:
# split trainind data to training and validation
num_val = 5000
indice = np.arange(x_train.shape[0])
np.random.shuffle(indice)
x_train = x_train[indice]/255
y_train = y_train[indice]
partial_train_x = x_train[:-num_val]
x_val = x_train[-num_val:]
partial_train_y = y_train[:-num_val]
y_val = y_train[-num_val:]
print(partial_train_x.shape,partial_train_y.shape)
print(x_val.shape,y_val.shape)

(45000, 32, 32, 3) (45000, 10)
(5000, 32, 32, 3) (5000, 10)


In [22]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256))
    model.add(layers.Dense(10,activation='softmax'))

    model.compile(loss='categorical_crossentropy',
           optimizer=optimizers.RMSprop(learning_rate=1e-4),
           metrics=['acc'])
    return model

In [23]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 128)        

In [ ]:
# start training
history = model.fit(
      partial_train_x,partial_train_y,
      epochs=80,
      batch_size=32,
      validation_data=(x_val,y_val),
      #verbose = 0
      )

Epoch 1/80
 995/1407 [====================>.........] - ETA: 15s - loss: 2.2326 - acc: 0.3092